# Orientation & Strain Mapping in PyCrystEM

PyCrystEM is an open-source library for crystallographic electron microscopy. Multi-dimensional data processing tools build on the HyperSpy library.

This Notebook provides an illustrative example of orientation and strain mapping using PyCrystEM. It is a trivial example used for code testing. Simple datasets are simulated and then the orientation or strain is mapped to illustrate the code syntax and to act as a sanity check.

## Authors

08/06/17 Duncan Johnstone - Developed for Trondheim Diffraction Workshop

## Requirements

Pycrystem 0.1

HyperSpy 1.3

PyMatGen

transforms3d

# Contents

1. <a href='#spe'> Specimen & Data</a>
2. <a href='#spe'> Loading & Inspection</a>
3. <a href='#loa'> Preprocessing for Indexation </a>
4. <a href='#vdf'> Orientation Mapping</a>
5. <a href='#ml'> Strain Mapping</a>

## <a id='loa'></a> 1. Import Requirements

In [1]:
%matplotlib tk
import numpy as np
import hyperspy.api as hs
import pycrystem as pc
import pymatgen as pmg
from pycrystem.indexation_generator import IndexationGenerator
from scipy.constants import pi

C:\Users\Duncan\AppData\Local\conda\conda\envs\pycrystem\lib\site-packages\pymatgen\util\plotting_utils.py:3: UserWarning: pymatgen.util.plotting_utils has been renamed pymatgen.util.plotting. This stub will be removed in PMG 5.0.
  warnings.warn("pymatgen.util.plotting_utils has been renamed "


## <a id='loa'></a> 2. Simulate Orientation Test Data

Specify the graphite crystal structure from a .cif file

In [ ]:
graphite = pmg.Structure.from_file('./graphite.cif')

Specify the silicon crystal structure by defining a lattice and atomic coordinates

In [ ]:
si = pmg.Element("Si")
lattice = pmg.Lattice.cubic(5.431)
silicon = pmg.Structure.from_spacegroup("Fd-3m",lattice, [si], [[0, 0, 0]])

Simulate a test dataset, which is a bi-crystal of graphite and silicon both rotated through an angle of 45 degrees about the axis out of the page across the navigation region.

In [ ]:
from pymatgen.transformations.standard_transformations import RotationTransformation

size = 256

radius=1.5
ediff = pc.ElectronDiffractionCalculator(300., 0.025)

rotaxis = [0, 0, 1]
thetas = np.arange(0, 45, 1)

data_graphite = []
for theta in thetas:
    rot = RotationTransformation(rotaxis, theta)
    greg = rot.apply_transformation(graphite)
    diff_dat = ediff.calculate_ed_data(greg, radius)
    dpi = diff_dat.as_signal(256, 0.03, 1.2)
    data_graphite.append(dpi.data)
    
data_silicon = []
for theta in thetas:
    rot = RotationTransformation(rotaxis, theta)
    sieg = rot.apply_transformation(silicon)
    diff_dat = ediff.calculate_ed_data(sieg, radius)
    dpi = diff_dat.as_signal(256, 0.03, 1.2)
    data_silicon.append(dpi.data)
    
data = [data_silicon] * 3 + [data_graphite] * 3
test_data = pc.ElectronDiffraction(data)
test_data.set_calibration(1.2/128)

Plot the test data to see what the model looks like.

In [ ]:
test_data.plot()

## <a id='loa'></a> 3. Perform Orientation Mapping

Define a list of rotations, here between 0-120 degrees about the axis out of the page, to include in the template bank used for pattern matching based orientation mapping.

In [2]:
rot_list = []
nstep=119
for i in np.arange(nstep):
    theta = (i*59.5/(nstep-1))/180*pi
    rot_list.append((theta, 0., 0.))

Simulate a library of diffraction pattern templates (to be used in pattern matching) for the two crystal structures (graphite and silicon) and all rotations specified in the list above.

In [3]:
edc = pc.ElectronDiffractionCalculator(50, 5e-2)
diff_gen = pc.DiffractionLibraryGenerator(edc)
struc_lib = dict()
struc_lib["gr"] = (graphite, rot_list)
struc_lib['si'] = (silicon, rot_list)
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=1.2/128,
                                            reciprocal_radius=1.,
                                            representation='euler')

NameError: name 'graphite' is not defined

Perform pattern matching between each diffraction pattern in the test data and the library of template patterns.

The indexer returns the 'n' best correlated orientations with a default value of the 5 best for each phase

In [ ]:
indexer = IndexationGenerator(test_data, library)
match_results = indexer.correlate()

Look at what's in the matching results:

column 1   = phase id (0 = graphite, 1 = silicon)
column 2-4 = Euler angles in the zxz convention (radians)
colum 5    = Correlation scores associated with the orientation and phase (top 5 for each phase, max = 1.)

In [ ]:
match_results.inav[0,0].data

Get the results for just one phase and look at it.

In [ ]:
phase_res = match_results.get_phase_results(phaseid=0)
phase_res.inav[0,0].data

Obtain a 'final' crystallographic map by returning the best matching phase and orientation for each pixel.

The crystallographic map results are:

column 1   = phase id
column 2-4 = Euler angles
column 5   = correlation score
column 6   = difference between this correlation score and the 2nd best correlation score

In [ ]:
cryst_map = match_results.get_crystallographic_map()
cryst_map.inav[0,0].data

Get a phase map from the results i.e. phase i.d at each pixel and plot it

In [ ]:
cryst_map.get_phase_map().plot()

Get an orientation map i.e. rotation angle at each pixel and plot it

In [ ]:
cryst_map.get_orientation_image().plot()

## <a id='loa'></a> 4. Simulate Strain Test Data

Define silicon crystal structure

In [ ]:
si = pmg.Element("Si")
lattice = pmg.Lattice.cubic(5.431)
structure = pmg.Structure.from_spacegroup("Fd-3m",lattice, [si], [[0, 0, 0]])
ediff = pc.ElectronDiffractionCalculator(300., 0.025)

Define a series of affine transformations from 0-2%, apply these to the crystal structure and simulate the diffraction in each case.

In [ ]:
from pymatgen.transformations.standard_transformations import DeformStructureTransformation

affines = [[[1, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.002, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.004, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.006, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.008, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.01, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.012, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.014, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.016, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.018, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.02, 0, 0], [0, 1, 0], [0, 0,  1]]]

data = []
for affine in affines:
    deform = DeformStructureTransformation(affine)
    strained = deform.apply_transformation(structure)
    diff_dat = ediff.calculate_ed_data(strained, 2.5)
    dpi = diff_dat.as_signal(512, 0.02, 2.5)
    data.append(dpi.data)
strain_series = pc.ElectronDiffraction(data)

#stack multiple versions of tilt series together so that you have repeats of measurements for error analysis
dp = hs.stack((strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series))

Plot the test data to see what it looks like.

In [ ]:
dp.plot()

## <a id='loa'></a> 5. Perform Strain Mapping

Create a model to the data which comprises a distorted version of a reference (unstrained) diffraction pattern at each probe position.

In [ ]:
m = dp.create_model()
ref = pc.ScalableReferencePattern(dp.inav[0,0])
m.append(ref)

Print the affine transform values associated with the distorted diffraction pattern before fitting.

In [ ]:
m.print_current_values()

Perform fitting

In [ ]:
m.multifit()

Construct the displacement graident tensor at each pixel from the fitting results

In [ ]:
disp_grad = ref.construct_displacement_gradient()

Perform (right) polar decomposition on the displacement gradient tensor to get rotation matrix, R, and strain matrix, U, at each pixel.

In [ ]:
R, U = disp_grad.polar_decomposition()

Get a strain map and plot it.

In [ ]:
strain_map = disp_grad.get_strain_maps()
strain_map.plot()